In [16]:
import yaml, os
from llama_index import Prompt
from llama_index import ServiceContext
from llama_index.llm_predictor import LLMPredictor
from llama_index import set_global_service_context
from llama_index.llms import AzureOpenAI, ChatMessage
from llama_index.embeddings import HuggingFaceEmbedding

In [2]:
with open('/Users/1zuu/Desktop/LLM RESEARCH/LlamaIndex Cooking/cadentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

os.environ['AD_OPENAI_API_KEY'] = credentials['AD_OPENAI_API_KEY']
os.environ['HUGGINGFACEHUB_API_TOKEN'] = credentials['HUGGINGFACEHUB_API_TOKEN']

In [3]:
!mkdir -p 'data/'
!wget 'https://github.com/run-llama/llama_docs_bot/blob/main/docs/getting_started/starter_example.md' -O 'data/starter_example.md'

--2023-12-13 15:30:39--  https://github.com/run-llama/llama_docs_bot/blob/main/docs/getting_started/starter_example.md
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20839 (20K) [text/plain]
Saving to: ‘data/starter_example.md’

data/starter_exampl 100%[===================>]  20.35K  --.-KB/s    in 0.05s   

2023-12-13 15:30:40 (382 KB/s) - ‘data/starter_example.md’ saved [20839/20839]



In [10]:
embedding_llm = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
llm=AzureOpenAI(
                deployment_name=credentials['AD_DEPLOYMENT_ID'],
                model=credentials['AD_ENGINE'],
                api_key=credentials['AD_OPENAI_API_KEY'],
                api_version=credentials['AD_OPENAI_API_VERSION'],
                azure_endpoint=credentials['AD_OPENAI_API_BASE'],
                max_tokens=5000
                )
chat_llm = LLMPredictor(llm)

service_context = ServiceContext.from_defaults(
                                                embed_model=embedding_llm,
                                                llm_predictor=chat_llm
                                                )
set_global_service_context(service_context)

# Define 2 Templates

In [5]:
text_qa_template = Prompt(
                        "Context information is below.\n"
                        "---------------------\n"
                        "{context_str}\n"
                        "---------------------\n"
                        "Given the context information and not prior knowledge, "
                        "answer the question: {query_str}\n"
                        )

refine_template = Prompt(
                        "We have the opportunity to refine the original answer "
                        "(only if needed) with some more context below.\n"
                        "------------\n"
                        "{context_msg}\n"
                        "------------\n"
                        "Given the new context, refine the original answer to better "
                        "answer the question: {query_str}. "
                        "If the context isn't useful, output the original answer again.\n"
                        "Original Answer: {existing_answer}"
                        )

In [6]:
with open("data/starter_example.md", "r") as f:
    context = f.read()

# Text QA Template Testing

In [7]:
question = "How can I install llama-index?"
prompt = text_qa_template.format(context_str=context, query_str=question)
response = llm.complete(prompt)
print(response.text)

You can install llama-index by following the installation steps provided in the installation.md file.


In [11]:
question = "How do I create an index?"
prompt = text_qa_template.format(context_str=context, query_str=question)
response = llm.complete(prompt)
print(response.text)

To create an index using LlamaIndex, you need to create a new `.py` file and import `VectorStoreIndex` and `SimpleDirectoryReader` from `llama_index`. Then, you can load the data using `SimpleDirectoryReader('data').load_data()` and build the index using `VectorStoreIndex.from_documents(documents)`.


In [13]:
question = "How do I create an index? Write your answer using only code."
prompt = text_qa_template.format(context_str=context, query_str=question)
response_gen = llm.stream_complete(prompt)
for response in response_gen:
    print(response.delta, end="")

To create an index, you can use the following code:

```python
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('data').load_data()
index = VectorStoreIndex.from_documents(documents)
```

This builds an index over the documents in the `data` folder (which in this case just consists of the essay text).

# Refine Template Testing

In [15]:
question = "How do I create an index? Write your answer using only code."
existing_answer = """To create an index using LlamaIndex, you need to follow these steps:

1. Download the LlamaIndex repository by cloning it from GitHub.
2. Navigate to the `examples/paul_graham_essay` folder in the cloned repository.
3. Create a new Python file and import the necessary modules: `VectorStoreIndex` and `SimpleDirectoryReader`.
4. Load the documents from the `data` folder using `SimpleDirectoryReader('data').load_data()`.
5. Build the index using `VectorStoreIndex.from_documents(documents)`.
6. To persist the index to disk, use `index.storage_context.persist()`.
7. To reload the index from disk, use the `StorageContext` and `load_index_from_storage` functions.

Note: This answer assumes that you have already installed LlamaIndex and have the necessary dependencies."""
prompt = refine_template.format(context_msg=context, query_str=question, existing_answer=existing_answer)
response = llm.complete(prompt)
print(response.text)

To create an index using LlamaIndex, follow these steps:

```python
from llama_index import VectorStoreIndex, SimpleDirectoryReader

# Load documents from the 'data' folder
documents = SimpleDirectoryReader('data').load_data()

# Build the index
index = VectorStoreIndex.from_documents(documents)

# Persist the index to disk
index.storage_context.persist()

# Reload the index from disk
from llama_index import StorageContext, load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir="./storage")
index = load_index_from_storage(storage_context)
```

Note that these steps assume that you have already installed LlamaIndex and have the necessary dependencies.


### Now we know how Completion Module works, let's see how Chat Module works

In [17]:
chat_history = [
    ChatMessage(role="system", content="You are a helpful QA chatbot that can answer questions about llama-index."),
    ChatMessage(role="user", content="How do I create an index?"),
]

response = llm.chat(chat_history)
print(response.message)

assistant: To create an index, you need to follow these steps:

1. Choose the content you want to index: Decide which pages or documents you want to include in your index.

2. Identify the keywords: Identify the keywords or phrases that best describe the content you want to index.

3. Create a list of keywords: Create a list of keywords or phrases that you want to include in your index.

4. Assign weights to the keywords: Assign weights to the keywords based on their importance.

5. Create the index: Use a software tool or a search engine to create the index.

6. Test the index: Test the index to make sure it returns accurate results.

7. Update the index: Update the index regularly to ensure that it remains accurate and up-to-date.

Keep in mind that the process of creating an index may vary depending on the software tool or search engine you use.
